In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("joint1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("joint2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2

In [6]:
x = MechanismState(Float64, mechanism);

In [7]:
rand!(x)
setdirty!(x)
x.q[joint1]

7-element Array{Float64,1}:
  0.306798
 -0.795949
 -0.350005
  0.387087
  0.400136
  0.959721
  0.142376

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
relative_transform(x, joint2.frameAfter, body1.frame)

Transform3D from "after_joint2" to "body1":
rotation: 1.7879338790737147 rad about [-0.30214160744162194,-0.9262890716763049,0.22516439493321777], translation: Vec(0.2670294977673644,0.3386398060824638,0.9719616586017255)


In [11]:
center_of_mass(x)

Point3D in "world": Vec(1.1825793413205135,0.5615072575759832,2.4599470074361625)

In [12]:
mass(mechanism)

0.6857156240727806

In [13]:
transform(x, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(1.5960437153594493,0.7631841043897759,2.3715515868973602)

In [14]:
relative_twist(x, body2, body1)

Twist of "after_joint2" w.r.t "after_joint1" in "world":
angular: Vec(0.03346700443095907,-0.11971582670001646,-0.024429090133831044), linear: Vec(0.2652683655123085,0.11835862324699253,-0.21661317863687235)

In [15]:
relative_twist(x, body1, world) + relative_twist(x, body2, body1) + relative_twist(x, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [16]:
p = path(mechanism, body2, world)
J = geometric_jacobian(x, p)

GeometricJacobian: body: "world", base: "after_joint2", expressed in "world":
6x7 Array{Float64,2}:
 -0.196664  -0.802101  -0.0854609  …  -0.0       -0.0        -0.0     
  0.703492  -0.414788   0.791748      -0.0       -0.0        -0.0     
  0.143554  -0.429634  -0.60484       -0.0       -0.0        -0.0     
 -1.55881   -0.26864   -1.55511       -0.802101  -0.0854609  -0.591042
 -0.695517  -0.38076    0.356897      -0.414788   0.791748    0.448425
  1.27289    0.869137   0.686915   …  -0.429634  -0.60484     0.67051 

In [17]:
Twist(J, velocity_vector(x, p))

Twist of "world" w.r.t "after_joint2" in "world":
angular: Vec(-0.6574288285000551,1.1078485112733094,0.17893420650871633), linear: Vec(-1.5059157713789495,-0.6245363390596695,1.2482704628793926)

In [18]:
mass_matrix(x)

7x7 Array{Float64,2}:
  2.92147      -0.934469    1.57084      …   1.17612       0.368275 
 -0.934469      0.5301      0.0921608       -0.47902      -0.105865 
  1.57084       0.0921608   7.27895          0.0           1.92944  
  5.55112e-17  -0.272958   -1.17612          2.77556e-17  -0.246225 
  0.272958      0.0         0.47902          5.55112e-17  -0.113861 
  1.17612      -0.47902    -8.32667e-17  …   0.685716      0.0137373
  0.368275     -0.105865    1.92944          0.0137373     0.744892 

In [19]:
A = momentum_matrix(x)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
  2.92313   -0.440744   5.99899   …   1.56842     0.498245   1.17542  
  2.30518   -1.42313   -3.30425      -0.346315    1.54071   -0.668256 
 -1.93456    0.658178  -3.45682      -0.674945   -0.591206  -0.923463 
  0.718461  -0.502061  -0.902426      0.0586019   0.405287  -0.199108 
 -0.743515   0.101585  -0.867102     -0.542914   -0.307492  -0.0181422
 -0.623502   0.203916  -0.215569  …   0.414748   -0.459779  -0.183865 

In [20]:
Momentum(A, velocity_vector(x))

Momentum expressed in "world":
angular: Vec(5.889558108862749,-3.5615187774559005,-3.2302076663913697), linear: Vec(-0.7904679438070757,-0.833797424434303,0.3028239911360766)

In [21]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(x, accel, world.frame)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(-0.3387362759202282,-0.7908612224929841,-0.28728934561778097), linear: Vec(1.4938512818417682,-0.9966154312373974,-1.023685655925873)

In [22]:
bias_acceleration(x, body2)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(0.005642474679423766,0.010071996225929436,-0.04162826466469219), linear: Vec(0.09645840888164293,0.0899544427748572,0.17050622290703588)

In [23]:
v̇ = [j::Joint => rand(num_velocities(j))::Vector{Float64} for j in joints(mechanism)]
τ = inverse_dynamics(x, v̇)

7-element Array{Float64,1}:
  1.98948  
  2.02722  
  0.0834082
  2.09249  
  4.09711  
 -2.56799  
 -1.33484  

In [24]:
rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [25]:
rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body9, Edge: joint9
  Vertex: body2, Edge: joint2
    Vertex: body3, Edge: joint3
      Vertex: body5, Edge: joint5
        Vertex: body8, Edge: joint8
  Vertex: body4, Edge: joint4
    Vertex: body6, Edge: joint6
  Vertex: body7, Edge: joint7
  Vertex: body10, Edge: joint10
  Vertex: body11, Edge: joint11